In [3]:
%load_ext autoreload


In [4]:
%autoreload 2

In [49]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation

In [135]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2015-06-04/"
path_sav = "/Volumes/friebelm/PhD/NU Plasma/Measurements/compilation/"
files_start = 1808
files_end = 1849

In [142]:
# cup configuration
cup_config = cycle_Sb
# Isotopes used for Interference correction
corr_isotopes_1 = {"Cd" : "111", "Te" : "125"}
corr_isotopes_2 = {"Cd" : "111","Te" : "125", "Xe" : "129"}
corr_isotopes_Sb = {"Te": "125"}
# Mass Range of cup configuration
mass_range = cycle_Sb_mass_range
#isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"], ["117", "118", "119", "122", "124"]]
#isotopes = [["112", "114", "115", "116", "117", "118", "119", "122", "124"]]
isotopes = [["116", "117", "118", "119", "122", "124"]]
data_sample_column = [(i + "/" + "120") for i in isotopes[0]]

#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = False

blk_corr = False

In [143]:
columns = ["cycle", "sample", "date", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)"]
blk_ls = []
sample_ls = []
df_all = pd.DataFrame()
files_1 = range(files_start, files_end, 1)
for sample in files_1:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    df_zero = df.data_zero_corr(sample)
    df_zero = pd.DataFrame(df_zero["cycle1"])
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    df_zero["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    df_zero["sample"] = sample_name
    df_zero["cycle"] = df_zero.index.values
    df_zero.index = df_zero["date"]
    df_zero = df_zero[columns]
    df_all = df_all.append(df_zero, ignore_index = True)

    if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean":
        blk_ls.append(sample)
    elif sample_name == "SQ" or sample_name == "teflon blank":
        None
    else:
        sample_ls.append(sample)

df_all

,cycle,sample,date,H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),Ax (1),L1 (1),L2 (1),L3 (1)
0,1,SPEX 100ppb,2015-06-04 14:59:00,-5.929000e-07,2.390442,5.527639,1.866145,7.207432,12.807256,3.331677,9.274943,2.899816,5.424166
1,2,SPEX 100ppb,2015-06-04 14:59:00,8.449800e-06,2.389681,5.529529,1.865456,7.210071,12.805200,3.330715,9.272399,2.898972,5.423426
2,3,SPEX 100ppb,2015-06-04 14:59:00,9.282800e-06,2.473279,5.720230,1.930785,7.459041,13.251494,3.447274,9.596558,3.000273,5.612432
3,4,SPEX 100ppb,2015-06-04 14:59:00,-2.174510e-05,2.480942,5.738638,1.936756,7.482604,13.292799,3.457866,9.626477,3.009814,5.629829
4,5,SPEX 100ppb,2015-06-04 14:59:00,1.347230e-05,2.506781,5.797064,1.956942,7.558470,13.430446,3.493868,9.725806,3.040728,5.687919
5,6,SPEX 100ppb,2015-06-04 14:59:00,-1.605510e-05,2.507173,5.798552,1.957304,7.560283,13.431487,3.494139,9.726926,3.041039,5.688064
6,7,SPEX 100ppb,2015-06-04 14:59:00,-1.022440e-05,2.503092,5.788864,1.953916,7.548192,13.410841,3.488490,9.711299,3.036238,5.679905
7,8,SPEX 100ppb,2015-06-04 14:59:00,-1.255700e-05,2.452258,5.671370,1.914436,7.394885,13.138214,3.417768,9.514619,2.974673,5.564382
8,9,SPEX 100ppb,2015-06-04 14:59:00,-1.706630e-05,2.493189,5.758736,1.945983,7.508453,13.356675,3.474209,9.671032,3.023516,5.655932
9,10,SPEX 100ppb,2015-06-04 14:59:00,-2.681080e-05,2.493701,5.767576,1.946744,7.519676,13.359145,3.475118,9.674409,3.024615,5.657467


In [144]:
files_1 = range(files_start, files_end, 1)
df_raw_signals = pd.DataFrame()
df_external_norm = pd.DataFrame()
df_external_norm_mean = pd.DataFrame()

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    # arbitrary blank positions
    #blk1 = [item for item in blk_ls if item < sample]
    #blk2 = [item for item in blk_ls if item > sample]
    #blk1 = blk1[-1]
    #blk2 = blk2[0]
    #blk_corr_sample == True
    
    # blank measurement surrounds sample
    #if ((sample-1) in blk_ls) and ((sample+1) in blk_ls):
    #    blk1 = sample-1
    #    blk2 = sample+1
    #    blk_corr_sample = True
    #else:
    #    blk_corr_sample = False
    
    if (blk_corr == True) and (blk_corr_sample == True):
        blk_1 = NU_data_read(path, blk1, cup_config)
        blk_2 = NU_data_read(path, blk2, cup_config)
        df_zero = df.data_zero_corr(sample)
        df_bgd_1 = blk_1.data_zero_corr(blk1)
        df_bgd_2 = blk_2.data_zero_corr(blk2)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_Sb , denom_corr_ratio)
        new_corr.data_bgd_corr(df_bgd_1, df_bgd_2)
    else:
        df_zero = df.data_zero_corr(sample)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_Sb , denom_corr_ratio)
    
    data_sample_raw = new_corr.raw_signals("120")
    data_sample_raw = pd.DataFrame.from_dict(data_sample_raw, orient = 'index')
    data_sample_raw = data_sample_raw[np.abs(data_sample_raw-data_sample_raw.mean())<=(2*data_sample_raw.std())]
    
    data_sample = new_corr.raw_ratios("120")
    data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')
    data_sample = data_sample[np.abs(data_sample-data_sample.mean())<=(2*data_sample.std())]
    
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_sample = data_sample[["116", "117", "118", "119", "122", "124"]]
    data_sample.columns = ["116/120", "117/120", "118/120", "119/120", "122/120", "124/120"]
    data_sample["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_sample["sample"] = sample_name
    data_sample["cycle"] = data_sample.index.values
    
    
    data_sample_raw["cycle"] = data_sample_raw.index.values
    data_sample = data_sample.merge(data_sample_raw)
    
    df_external_norm = df_external_norm.append(data_sample)

In [145]:
df_external_norm.set_index("date", inplace = True)
columns = ["cycle", "sample"]
columns.extend(data_sample_column)
columns.extend(sorted(data_sample_raw.drop("cycle", axis=1).columns))
df_external_sub = pd.DataFrame(df_external_norm, columns = columns)
df_external_sub = df_external_sub[df_external_sub["120"] > 4]
df_external_sub

,cycle,sample,116/120,117/120,118/120,119/120,122/120,124/120,116,117,118,119,120,122,124
date,,,,,,,,,,,,,,,
2015-06-04 14:59:00,1,SPEX 100ppb,0.423523,0.226420,0.724194,0.260140,0.145710,0.186648,5.424166,2.899816,9.274943,3.331677,12.807256,1.866145,2.390442
2015-06-04 14:59:00,2,SPEX 100ppb,0.423533,0.226390,0.724112,0.260106,NaN,0.186618,5.423426,2.898972,9.272399,3.330715,12.805200,1.865456,2.389681
2015-06-04 14:59:00,3,SPEX 100ppb,0.423532,0.226410,0.724187,0.260142,0.145703,0.186642,5.612432,3.000273,9.596558,3.447274,13.251494,1.930785,2.473279
2015-06-04 14:59:00,4,SPEX 100ppb,0.423525,0.226424,0.724187,0.260131,0.145700,0.186638,5.629829,3.009814,9.626477,3.457866,13.292799,1.936756,2.480942
2015-06-04 14:59:00,5,SPEX 100ppb,0.423509,0.226406,0.724161,0.260145,0.145709,0.186649,5.687919,3.040728,9.725806,3.493868,13.430446,1.956942,2.506781
2015-06-04 14:59:00,6,SPEX 100ppb,0.423487,0.226411,0.724188,0.260145,0.145725,0.186664,5.688064,3.041039,9.726926,3.494139,13.431487,1.957304,2.507173
2015-06-04 14:59:00,7,SPEX 100ppb,0.423531,0.226402,0.724138,0.260125,0.145697,0.186647,5.679905,3.036238,9.711299,3.488490,13.410841,1.953916,2.503092
2015-06-04 14:59:00,8,SPEX 100ppb,0.423527,0.226414,0.724194,0.260139,0.145715,0.186651,5.564382,2.974673,9.514619,3.417768,13.138214,1.914436,2.452258
2015-06-04 14:59:00,9,SPEX 100ppb,0.423454,0.226367,0.724060,0.260110,0.145694,0.186662,5.655932,3.023516,9.671032,3.474209,13.356675,1.945983,2.493189


In [146]:
def SE(x):
    SE = np.std(x) / x.count()
    return SE

df_external_norm_mean = df_external_sub.drop("cycle", axis=1).groupby(df_external_sub.index).mean()
df_external_norm_mean_sd = df_external_sub.drop("cycle", axis=1).groupby(df_external_sub.index).agg([np.mean, SE])
df_external_norm_mean["sample"] = df_external_norm[df_external_norm["cycle"] == 1]["sample"]
df_external_norm_mean_sd["sample"] = df_external_norm[df_external_norm["cycle"] == 1]["sample"]
#df_external_norm_mean = df_external_norm_mean[1:]
#df_external_norm_mean = df_external_norm_mean.drop(datetime.strptime("20150310 23:59", "%Y%m%d %H:%M"))
#df_external_norm_mean_sd = df_external_norm_mean_sd[1:]
df_external_norm_mean

,116/120,117/120,118/120,119/120,122/120,124/120,116,117,118,119,120,122,124,sample
date,,,,,,,,,,,,,,
2015-06-04 14:59:00,0.423495,0.226397,0.724149,0.260134,0.145712,0.186658,5.588835,2.987823,9.556677,3.433016,13.197125,1.922959,2.463355,SPEX 100ppb
2015-06-04 15:08:00,0.423375,0.226352,0.724047,0.260114,0.145727,0.186707,5.515881,2.948992,9.433048,3.388820,13.028356,1.898569,2.432512,SPEX 100ppb
2015-06-04 15:16:00,0.423525,0.226415,0.724175,0.260140,0.145704,0.186644,5.497169,2.939128,9.400645,3.376918,12.979414,1.891408,2.422849,SPEX 100ppb
2015-06-04 15:33:00,0.423580,0.226437,0.724221,0.260147,0.145693,0.186624,5.526597,2.954791,9.450370,3.394665,13.047206,1.901158,2.435267,NIST 100ppb
2015-06-04 15:42:00,0.423639,0.226446,0.724255,0.260152,0.145688,0.186606,5.487667,2.933469,9.382131,3.370063,12.954192,1.887280,2.417391,NIST 100ppb
2015-06-04 15:51:00,0.423659,0.226461,0.724277,0.260153,0.145679,0.186591,5.384537,2.878305,9.205467,3.306522,12.709637,1.851419,2.371519,NIST 100ppb
2015-06-04 15:59:00,0.423579,0.226434,0.724208,0.260144,0.145694,0.186628,5.469396,2.923526,9.350413,3.358782,12.912348,1.881108,2.409814,NIST 100ppb
2015-06-04 16:08:00,0.423578,0.226435,0.724218,0.260145,0.145691,0.186621,5.519929,2.950204,9.436642,3.389724,13.031293,1.898378,2.431490,NIST 100ppb
2015-06-04 16:16:00,0.423519,0.226409,0.724148,0.260136,0.145702,0.186650,5.406760,2.890424,9.244743,3.321003,12.766162,1.860086,2.382836,NIST 100ppb


In [119]:
#sub_index = df_external_norm_mean[(df_external_norm_mean.sample.shift(1) == df_external_norm_mean.sample.shift(-1)) & (df_external_norm_mean.sample.shift(1) == "NIST 100ppb")].index
sample = df_external_norm_mean["sample"]
new = df_external_norm_mean[df_external_norm_mean.columns[df_external_norm_mean.dtypes != 'object']]

new =  ((new / ((new.shift(1) + new.shift(-1))/2))-1)*1000
new["sample"] = sample

new = new[new.index.isin(sub_index)]
#df_external_norm_signal = df_external_norm_signal[df_external_norm_signal['sample'].isin(sub_index.tolist())]
#df_new_signal = new.merge(df_external_norm_signal, left_index=True, right_index=True)
#new[new.sample != "NIST 100ppb"]


In [123]:
new.groupby("sample").agg([np.mean, np.std, np.size])

116/120                     117/120                 \
                        mean       std size         mean       std size   
sample                                                                    
BHVO-2 1_1(2) I    -0.562374       NaN    1    -0.435635       NaN    1   
BHVO-2 1_2(1) I    -0.761249       NaN    1    -0.576920       NaN    1   
BHVO-2 2-2 I       -0.630289       NaN    1    -0.514774       NaN    1   
BHVO-2 2_1 I       -0.638599       NaN    1    -0.507914       NaN    1   
J1 1_1 I           -0.479768  0.001827    2    -0.366074  0.020097    2   
J2 1_1 I           -0.337210       NaN    1    -0.287974       NaN    1   
NIST 100ppb         0.005390  0.013163    3    -0.011433  0.036562    3   
SPEX 100ppb        -0.280679  0.015528   45    -0.208626  0.040104   45   
SPEX1 1_1(1) I   3826.688675       NaN    1  2233.664489       NaN    1   
SPEX1 2_1 I        -0.385132       NaN    1    -0.306317       NaN    1   
SPEX1 II 3_1 I     -0.411635       NaN    1    -0.305280       NaN    1   
SPEX1 II 4_1 I     -0.522829       NaN    1    -0.372163       NaN    1   

                     118/120                    119/120                 \
                        mean       std size        mean       std size   
sample                                                                   
BHVO-2 1_1(2) I    -0.293450       NaN    1   -0.142375       NaN    1   
BHVO-2 1_2(1) I    -0.389528       NaN    1   -0.207285       NaN    1   
BHVO-2 2-2 I       -0.213156       NaN    1   -0.087376       NaN    1   
BHVO-2 2_1 I       -0.219054       NaN    1   -0.113395       NaN    1   
J1 1_1 I           -0.219902  0.019291    2   -0.079692  0.016256    2   
J2 1_1 I           -0.178934       NaN    1   -0.028617       NaN    1   
NIST 100ppb         0.025868  0.023632    3   -0.024272  0.032934    3   
SPEX 100ppb        -0.138885  0.016831   45   -0.073155  0.019215   45   
SPEX1 1_1(1) I   1177.724175       NaN    1  482.421221       NaN    1   
SPEX1 2_1 I        -0.206499       NaN    1   -0.082116       NaN    1   
SPEX1 II 3_1 I     -0.205526       NaN    1   -0.098153       NaN    1   
SPEX1 II 4_1 I     -0.246235       NaN    1   -0.120331       NaN    1   

                    122/120                    124/120                 
                       mean       std size        mean       std size  
sample                                                                 
BHVO-2 1_1(2) I    0.234118       NaN    1    0.529079       NaN    1  
BHVO-2 1_2(1) I    0.359455       NaN    1    0.720012       NaN    1  
BHVO-2 2-2 I       0.293040       NaN    1    0.485557       NaN    1  
BHVO-2 2_1 I       0.297594       NaN    1    0.478529       NaN    1  
J1 1_1 I           0.306541  0.080122    2    0.544771  0.074697    2  
J2 1_1 I           0.154632       NaN    1    0.344715       NaN    1  
NIST 100ppb       -0.032520  0.034205    3   -0.028265  0.011089    3  
SPEX 100ppb        0.131179  0.016331   45    0.259360  0.017111   45  
SPEX1 1_1(1) I  -527.178495       NaN    1 -784.334288       NaN    1  
SPEX1 2_1 I        0.167485       NaN    1    0.355228       NaN    1  
SPEX1 II 3_1 I     0.195785       NaN    1    0.367522       NaN    1  
SPEX1 II 4_1 I     0.288305       NaN    1    0.490732       NaN    1

In [80]:
df_all.to_csv(path_sav + "Sn_H6_L3_Sb_NU_Signals_zero_corr.csv", mode='a', header=False)

In [100]:
df_external_sub.to_csv(path_sav + "Sn_H6_L3_SB_external_norm.csv", mode='a', header=False)

In [147]:
df_external_norm_mean_sd.to_csv(path + "Sn_H6_L3_SB_external_norm_mean_raw.csv", mode='a', header=True)